In [ ]:
#!pip install pandas-datareader

In [ ]:
#!pip install pandas-bokeh

In [ ]:
import datetime

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas_datareader as pdr
import pandas_bokeh

pandas_bokeh.output_notebook()

In [ ]:
shares = {'A', 'B', 'C', 'D', 'E', 'F'}
shares

In [ ]:
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols

symbols = get_nasdaq_symbols()
symbols.loc[shares]

In [ ]:
names = symbols.loc[shares]['Security Name'].to_dict()
names

In [ ]:
df = pd.DataFrame(data=None)

for s in shares:
    st = pdr.get_data_yahoo(s).reset_index()
    st['Symbol'] = s    
    df = df.append(st, ignore_index=True)
    
df['Names'] = df['Symbol'].map(names)
    
df.info()

In [ ]:
df.sample(10)

In [ ]:
df['Names'] = df['Names'].str.extract("(\w+)")
df.sample(10)

In [ ]:
pvt = df.pivot(index='Date', columns='Names', values='Close')
pvt.head()

In [ ]:
pvt.plot_bokeh(kind='line', line_width=.75, legend='top_left');

In [ ]:
pvt.plot_bokeh(kind='area', line_width=.75, legend='top_left');

In [ ]:
pvt.plot_bokeh(kind='area', stacked=True, normed=100, line_width=.75, legend=False);

In [ ]:
pvt.tail(20)

In [ ]:
pvt.tail(20).plot_bokeh(kind='point', line_width=.75, ylim=(0,120), legend=False);

In [ ]:
sym = ['A', 'D', 'E']

df.query("Symbol in @sym").plot_bokeh(kind='scatter', x='Close', y='Volume', category='Symbol', line_color='white', alpha=.4, disable_scientific_axes=True);

In [ ]:
plots = []

colors = ['red', 'orange', 'yellow', 'green', 'blue', 'violet']

for i, s in enumerate(sorted(shares)):
    
    p = df.query("Symbol == @s").plot_bokeh(kind='scatter',
                                            x='Close',
                                            y='Volume',
                                            category='Names',
                                            alpha=.3,
                                            color=colors[i],
                                            line_width=0,
                                            title=f"Chart for: {s}",
                                            disable_scientific_axes=True,
                                            hovertool=False,
                                            legend=False,
                                            show_figure=False);
    plots.append(p)
    
pandas_bokeh.plot_grid([plots[:3], plots[3:]], plot_width=400);

In [ ]:
grp = df.groupby('Names').mean()['Close']
grp

In [ ]:
grp.reset_index()

In [ ]:
grp.plot_bokeh(kind='bar', color='blue', line_color='gray', legend=False);

In [ ]:
np.histogram(df['Close'], 12)

In [ ]:
np.linspace(1, 100, 12)

In [ ]:
df['Close'].plot_bokeh(kind='hist', bins=np.linspace(1, 100, 12), color='pink', line_color='red', line_width=.5, legend=False);

In [ ]:
xf = df[df['Date'] == df['Date'].max()].copy()
xf

In [ ]:
xf['Close'].values

In [ ]:
np.random.randint(1,100, len(xf))

In [ ]:
cnt = 0
lns = len(xf)

while True:
    cnt += 1
    try_shares = np.random.randint(1,100,lns)
    #total_value = (xf['Close'].values * try_shares).sum()
    total_value = np.dot(xf['Close'].values, try_shares).round(2)
    if (total_value >= 27000) & (total_value <= 30000) & (cnt <= 1_000_000):
        print(try_shares)
        break
    continue

In [ ]:
print(f"Number of tries: {cnt:,}")

In [ ]:
(xf['Close'].values * try_shares).sum().round(2)

In [ ]:
xf['Shares'] = try_shares
xf

In [ ]:
from pandas_datareader import wb

In [ ]:
country_data = wb.get_countries()
country_data

In [ ]:
country_data.dropna(subset=['longitude', 'latitude'], inplace=True)
country_data

In [ ]:
country_data.plot_bokeh.map(x='longitude', y='latitude', color='purple', hovertool_string="""@{name}""", legend=False, figsize=(1000, 800));

In [ ]:
indicator_df = wb.get_indicators()
names = indicator_df.name.str.contains('energy')
sources = indicator_df.source == "World Development Indicators"

indicators = indicator_df[names & sources]

indicators

In [ ]:
matches = wb.search("pop.*")
matches.head()

In [ ]:
start_year, end_year = 2005, 2015

country_codes = ['CHN', 'IND', 'USA', 'IDN', 'BRA', 'MEX', 'JPN']
indicator_codes = ['EG.USE.COMM.CL.ZS', 'EG.USE.COMM.FO.ZS']
df = wb.download(indicator=indicator_codes, country=country_codes, start=start_year, end=end_year).dropna()

type(df)

In [ ]:
df

In [ ]:
df.columns = ['alternative', 'fossil']

In [ ]:
df.reset_index(inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
df['year_end'] = df['year'].apply(lambda x: datetime.datetime(int(x), 12, 31))

In [ ]:
df.head()

In [ ]:
df.pivot_table(index='year_end', columns='country', values='alternative', fill_value=0).plot_bokeh(kind='line', line_width=.75, legend='bottom_left');